In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
print("ok")

ok


In [0]:
def submission(y_test_pred, sample,submission_name):
  #creates a submission file given Original test set for ID and the predictions
  test_id=(sample.to_numpy())[:,0]

  submission={"Id": test_id ,
              "y": y_test_pred.flatten()
             }
  submission=pd.DataFrame(submission)
  print(submission.head())

  submission.to_csv(submission_name)

In [0]:
train_eeg1=pd.read_csv("/content/drive/My Drive/Third year/Advanced Machine Learning/task4/train_eeg1.csv")
train_eeg2=pd.read_csv("/content/drive/My Drive/Third year/Advanced Machine Learning/task4/train_eeg2.csv")
train_emg=pd.read_csv("/content/drive/My Drive/Third year/Advanced Machine Learning/task4/train_emg.csv")
train_labels=pd.read_csv("/content/drive/My Drive/Third year/Advanced Machine Learning/task4/train_labels.csv")

test_eeg1=pd.read_csv("/content/drive/My Drive/Third year/Advanced Machine Learning/task4/test_eeg1.csv")
test_eeg2=pd.read_csv("/content/drive/My Drive/Third year/Advanced Machine Learning/task4/test_eeg2.csv")
test_emg=pd.read_csv("/content/drive/My Drive/Third year/Advanced Machine Learning/task4/test_emg.csv")

sample=pd.read_csv("/content/drive/My Drive/Third year/Advanced Machine Learning/task4/sample.csv")

In [0]:
train_eeg1_numpy=np.expand_dims(train_eeg1.to_numpy()[:,1:],-1)
train_eeg2_numpy=np.expand_dims(train_eeg2.to_numpy()[:,1:],-1)
train_emg_numpy=np.expand_dims(train_emg.to_numpy()[:,1:],-1)
train_labels_numpy=train_labels.to_numpy()[:,1:]

test_eeg1_numpy=np.expand_dims(test_eeg1.to_numpy()[:,1:],-1)
test_eeg2_numpy=np.expand_dims(test_eeg2.to_numpy()[:,1:],-1)
test_emg_numpy=np.expand_dims(test_emg.to_numpy()[:,1:],-1)

In [0]:

train_labels_numpy[np.where(train_labels_numpy==1)]=0
train_labels_numpy[np.where(train_labels_numpy==2)]=1
train_labels_numpy[np.where(train_labels_numpy==3)]=2

In [0]:
train_all_channels=np.concatenate((train_eeg1_numpy,train_eeg2_numpy,train_emg_numpy),axis=2)
train_all_channels.shape

(64800, 512, 3)

In [0]:
test_all_channels=np.concatenate((test_eeg1_numpy,test_eeg2_numpy,test_emg_numpy),axis=2)
test_all_channels.shape

(43200, 512, 3)

In [0]:
del train_eeg1
del train_eeg2
del train_emg

del test_eeg1
del test_eeg2
del test_emg

del train_eeg1_numpy
del train_eeg2_numpy
del train_emg_numpy

del test_eeg1_numpy
del test_eeg2_numpy
del test_emg_numpy




In [0]:
import gc
gc.collect()

22

In [0]:
#n defines the number of samples we append before and after each samples (2x)
n=2
n_times=512*(2*n+1)
augmented_train=np.empty((64800,n_times,3)) #512x2xn+512=1536
zeros=np.zeros((512,3))
for i in range(train_all_channels.shape[0]):
  row=np.empty((0,3))
  row2=np.empty((n_times,3))
  if i<1:
    for j in range(n-i):
      row=np.concatenate((row,zeros),axis=0)
    for j in range(i):
      row=np.concatenate((row,train_all_channels[j]),axis=0)
    for j in range(n+1):
      row=np.concatenate((row,train_all_channels[i+j]),axis=0)
    augmented_train[i]=row
    previous_row=row
    continue
  
  elif i>(64800-(n+1)):
    for j in range(n):
      row=np.concatenate((row,train_all_channels[i-(j+1)]),axis=0)
    for j in range(i,64800):
      row=np.concatenate((row,train_all_channels[j]),axis=0)
    for j in range(n-(64800-i)+1):
      row=np.concatenate((row,zeros),axis=0)
    augmented_train[i]=row
    previous_row=row
    continue
  
  row2[:n_times-512]=previous_row[512:]
  row2[n_times-512:]=train_all_channels[i+1]
  augmented_train[i]=row2
  previous_row=row2
  



In [0]:

augmented_test=np.empty((43200,n_times,3)) #512x2xn+512=1536
zeros=np.zeros((512,3))
for i in range(test_all_channels.shape[0]):
  row=np.empty((0,3))
  row2=np.empty((n_times,3))
  if i<1:
    for j in range(n-i):
      row=np.concatenate((row,zeros),axis=0)
    for j in range(i):
      row=np.concatenate((row,test_all_channels[j]),axis=0)
    for j in range(n+1):
      row=np.concatenate((row,test_all_channels[i+j]),axis=0)
    augmented_test[i]=row
    previous_row=row
    continue
  
  elif i>(43200-(n+1)):
    for j in range(n):
      row=np.concatenate((row,test_all_channels[i-(j+1)]),axis=0)
    for j in range(i,43200):
      row=np.concatenate((row,test_all_channels[j]),axis=0)
    for j in range(n-(43200-i)+1):
      row=np.concatenate((row,zeros),axis=0)
    augmented_test[i]=row
    previous_row=row
    continue
  
  row2[:(n_times-512)]=previous_row[512:]
  row2[n_times-512:]=test_all_channels[i+1]
  augmented_test[i]=row2
  previous_row=row2



In [0]:
augmented_train.shape

(64800, 2560, 3)

In [0]:
augmented_test.shape

(43200, 2560, 3)

In [0]:
del train_all_channels
del test_all_channels

In [0]:

gc.collect()

0

In [0]:
pip install -U mne

Requirement already up-to-date: mne in /usr/local/lib/python3.6/dist-packages (0.19.2)


In [0]:
from mne.time_frequency import tfr_stockwell
from mne import EpochsArray,create_info

import mne
from mne.time_frequency import psd_welch

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer

/usr/local/lib/python3.6/dist-packages/numba/decorators.py:146: RuntimeWarning: Caching is not available when the 'parallel' target is in use. Caching is now being disabled to allow execution to continue.
  warnings.warn(msg, RuntimeWarning)


In [0]:
def extract_freq_bands(epochs):
    # specific frequency bands
    FREQ_BANDS = {"delta": [0.5, 4.5],
                  "theta": [4.5, 8.5],
                  "alpha": [8.5, 11.5],
                  "sigma": [11.5, 15.5],
                  "beta": [15.5, 30]}

    psds, freqs = psd_welch(epochs, picks='eeg', fmin=0.5, fmax=30.)
    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    X = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_band = psds[:, :, (freqs >= fmin) & (freqs < fmax)].mean(axis=-1)
        X.append(psds_band.reshape(len(psds), -1))
    
    final=np.concatenate(X, axis=1)
    final=np.nan_to_num(final)

    return final

In [0]:
label2=augmented_train[np.where(train_labels_numpy==2)[0],:]
augmented_train=np.concatenate([augmented_train,label2,label2,label2,label2,label2,label2,label2,label2,label2,label2,label2])
two=np.expand_dims(2*np.ones(label2.shape[0]),-1)
train_labels_numpy_aug=np.concatenate([train_labels_numpy,two,two,two,two,two,two,two,two,two,two,two])

In [0]:
del label2

In [0]:
gc.collect()

0

In [0]:
augmented_train.shape

(103883, 2560, 3)

In [0]:
train_labels_numpy_aug.shape

(103883, 1)

In [0]:
ch_names=["eeg1","eeg2","emg"]
ch_types=["eeg","eeg","emg"]
sfreq=128
info=create_info(ch_names=ch_names,sfreq=sfreq,ch_types=ch_types)
epochs=EpochsArray(np.reshape(augmented_train,(103883,3,n_times)),info=info)

103883 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


In [0]:
augmented_train=None
del augmented_train

In [0]:
gc.collect()

0

In [0]:
epochs.shape

AttributeError: ignored

In [0]:
extracted=extract_freq_bands(epochs)

Effective window size : 2.000 (s)


In [0]:


pipe = make_pipeline(FunctionTransformer(extract_freq_bands, validate=False),
                     RandomForestClassifier(n_estimators=100, random_state=42,max_depth=15))

# Train

pipe.fit(epochs, train_labels_numpy_aug)

# Test
y_pred = pipe.predict(epochs)

# Assess the results
y_test = train_labels_numpy_aug
acc = accuracy_score(y_test,y_pred)

print("Accuracy score: {}".format(acc))

KeyboardInterrupt: ignored

In [0]:
epochs_test=EpochsArray(np.reshape(augmented_test,(43200,3,n_times)),info=info)
y_pred_test=pipe.predict(epochs_test)

43200 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
Effective window size : 2.000 (s)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()


In [0]:
y_pred_test=y_pred_test+1
submission(y_pred_test,sample,"augmented2_rf_md15.csv")